# Melting and pivotting dataframes with `pandas`
[Melting](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) and [pivotting](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot) are (in my opinion) two of the most important dataframe transformations. They are inverses of each other meaning that they undo what the other does. In an attempt to keep this brief, I won't spend much time here discussing why these operations are important, just how to perform them.

Some brief definitions:

**Melting**: Melting is an operation that will vertically concatenate (or stack) two or more columns of your dataframe on top of each other and then generate a new column to indicate the contents of this newly formed column. This creates what I call a **longform** dataframe.

**Pivotting**: Pivotting will take a column of information, break it up into it's consituent categories, and place the data for each category in its own column. This creates a **wideform** dataframe.
## Long and wideform data
**Wideform dataframe**

| Index | Feature 1 | Feature 2 |
|-------|-----------|-----------|
| I1    | V11       | V12       |
| I2    | V21       | V22       |

**Longform dataframe**

| Index | feature   | value |
|-------|-----------|-------|
| I1    | Feature 1 | V11   |
| I1    | Feature 2 | V12   |
| I2    | Feature 1 | V21   |
| I2    | Feature 2 | V12   |

These concepts are very complicated to describe, but I think they will be more obvious when seen in action.

In [2]:
import pandas as pd
import seaborn as sns

# Load

In [10]:
"""Load"""
df = sns.load_dataset('iris')
df['flower'] = [f'flower_{x}' for x in range(df.shape[0])]
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,flower
0,5.1,3.5,1.4,0.2,setosa,flower_0
1,4.9,3.0,1.4,0.2,setosa,flower_1
2,4.7,3.2,1.3,0.2,setosa,flower_2
3,4.6,3.1,1.5,0.2,setosa,flower_3
4,5.0,3.6,1.4,0.2,setosa,flower_4


# Melting

In [13]:
"""Melt"""
# Here we will melt the measurement of each flower so that each metric gets it's own row in our dataframe
melted = df.melt(['flower','species'], [c for c in df.columns if c not in ['flower','species']], 'metric', 'measurement')
melted.head()

,flower,species,metric,measurement
0,flower_0,setosa,sepal_length,5.1
1,flower_1,setosa,sepal_length,4.9
2,flower_2,setosa,sepal_length,4.7
3,flower_3,setosa,sepal_length,4.6
4,flower_4,setosa,sepal_length,5.0


As you can see, I have melted the above dataframe so now each `measurement` for each `flower` has it's own row. If you take a look at the [melting documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) you'll see the names of each of the arguments in the function. I also want to note that if you don't pass any `value_vars`, `melt` will melt on all columns not passed as `id_vars`, meaning that the following would produce the same results:
```python
df.melt(['flower', 'species'], value_name='metric', var_name='measurement')
```

# Pivotting

In [20]:
"""Pivot"""
# Here we will pivot the `melted` df by using the `pivot` function. `pivot` only accepts one index
#so as you will see, we will loose the `species` information when using `pivot`
melted.pivot('flower', 'metric', 'measurement').head()

metric,petal_length,petal_width,sepal_length,sepal_width
flower,,,,
flower_0,1.4,0.2,5.1,3.5
flower_1,1.4,0.2,4.9,3.0
flower_10,1.5,0.2,5.4,3.7
flower_100,6.0,2.5,6.3,3.3
flower_101,5.1,1.9,5.8,2.7


In order to keep all of the information, we will use a similar function called [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table). This function also pivots dataframes, but supports multi-indexing (you will see why this is relevant in a second). 

**Important**: `pivot_table` also will aggregate values (taking the mean of values) when pivotting. This is handy, but can be dangerous if you do not wish to aggregate values, which would happen if more than one value were to be apart of each group. This is a hard concept to convey over text... so I will try to illustrate it below:

In [24]:
# Ensure that each group is of size 1 before using pivot table to ensure not unwanted aggregation is occuring
melted.groupby(['flower', 'species', 'measurement']).size().unique()

array([1], dtype=int64)

In [26]:
# Now that we know each group is of size one, we can safely use `pivot_table`
pivotted = melted.pivot_table('measurement', ['flower', 'species'], 'metric')
pivotted.head()

,metric,petal_length,petal_width,sepal_length,sepal_width
flower,species,,,,
flower_0,setosa,1.4,0.2,5.1,3.5
flower_1,setosa,1.4,0.2,4.9,3.0
flower_10,setosa,1.5,0.2,5.4,3.7
flower_100,virginica,6.0,2.5,6.3,3.3
flower_101,virginica,5.1,1.9,5.8,2.7


In [28]:
# And if we `reset_index` we will get our original back
pivotted.reset_index().head()

metric,flower,species,petal_length,petal_width,sepal_length,sepal_width
0,flower_0,setosa,1.4,0.2,5.1,3.5
1,flower_1,setosa,1.4,0.2,4.9,3.0
2,flower_10,setosa,1.5,0.2,5.4,3.7
3,flower_100,virginica,6.0,2.5,6.3,3.3
4,flower_101,virginica,5.1,1.9,5.8,2.7
